<table>
<tr><td><img style="height: 150px;" src="images/geo_hydro1.jpg"></td>
<td bgcolor="#FFFFFF">
    <p style="font-size: xx-large; font-weight: 900; line-height: 100%">pyGROWAT1D</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);"><b style=color:red;>GRO</b>und<b style=color:red;>WAT</b>er</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Georg Kaufmann</p>
    </td>
<td><img style="height: 150px;" src="images/pyGROWAT1D.png"></td>
</tr>
</table>

----
# `pyGROWAT1D`

GROundWATer1D, a program package for  flow in porous rocks.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse.linalg
import sys

import libGROWAT1D

----
# Run `pyGROWAT1D`

Now we have assembed and discussed all relevant functions.

We group the functions in a wrapper function, `pyGROWAT1D()`.
Two options are passed:
- `state`: choice for 
    - **steady-state solution:** `'ss'`
    - **transient solution:** `'t'`
- `saved`: choice for output 
     - 'nnn': no,no,no
     - 'snn': screen,no,no
     - 'san': screen,ascii,no 
     - 'snv': screen,no,plot ...

In [2]:
def pyGROWAT1D(state='ss',saved='snn',path='work/'):
    eastmin,eastmax,nx,time_start,time_end,time_step,time_scale,whichtime = \
    libGROWAT1D.readParameter1D(path=path,control=True)
    dataBC = libGROWAT1D.readHEADBC1D(path=path,control=True)
    dataMAT = libGROWAT1D.readMaterials1D(path=path,control=True)
    X,dx = libGROWAT1D.createNodes1D(eastmin,eastmax,nx,control=True)
    K,S,head,flow,vx = libGROWAT1D.createFields1D(nx)
    K,S = libGROWAT1D.createProperties1D(dataMAT,K,S,X,nx,control=True)

    itime    = 0
    time     = time_start
    while (time <= time_end):
        # set boundary conditions
        ibound,irecharge,head,flow = libGROWAT1D.buildHEADBC1D(dataBC,dx,time,time_scale,head,flow,control=False)
        # assemble matrix and rhs of system of equations
        headOld = np.copy(head)
        flowOld = np.copy(flow)
        if (state=='ss'):
            matrix,rhs = libGROWAT1D.buildHeadEquations1D_ss(dx,nx,K,head,flow,ibound)
        if (state=='t'):
            matrix,rhs = libGROWAT1D.buildHeadEquations1D_t(dx,nx,time_step,K,S,head,headOld,flow,flowOld,ibound,omega=1.1)
        #print('Sparse CSR matrix: ',sys.getsizeof(matrix),' bytes')
        # sparse matrix solver for system of equations
        head = libGROWAT1D.solveLinearSystem1D(matrix,rhs)
        # calculate velocities
        xc,vcx = libGROWAT1D.solveVelocities1D(nx,time_scale,X,K,head)
        vabs = np.sqrt(vcx**2)
        # control output to screen
        if (saved[0]=='s'): libGROWAT1D.saveToScreen1D(saved,time,time_scale,head,vabs) 
        # save to file
        if (saved[1]=='a'): libGROWAT1D.saveHeadsAndVelocities1D(itime,time,time_scale,whichtime,X,head,flow,xc,vcx,ibound,irecharge,path=path)
        # plot to file
        if (saved[2]=='v'): libGROWAT1D.plotHeadsAndVelocities1D(itime,time,time_scale,X,head,xc,vcx,ibound,irecharge,vmin=0.4,vmax=4.4,path=path)  
        time += time_step
        itime += 1
    return

In [5]:
pyGROWAT1D(state='t',saved='snv',path='work/runREF/')

== GROWAT1D ==
               path:         work/runREF/
      xmin,xmax [m]:       0.00    2000.00
                 nx:         41
    time_start[day]:       0.00
      time_end[day]:     150.00
     time_step[day]:       2.00
          whichtime: day
                 BC: [  3.      0.     40.     50.    150.      0.822]
                 BC: [  4.  20.  20. 100. 150.  -1.]
                 BC: [  1.   0.   0.   0. 150.   1.]
                 BC: [  2.  40.  40.   0. 150.   3.]
          Materials: ['Soil1' '1000' '2000' '3e-3' '1e-2']
          Materials: ['Soil2' '1000' '2000' '3e-3' '1e-2']
    X.min,X.max [m]:       0.00    2000.00
  K.min,K.max [m/s]:   0.003000   0.003000
  S.min,S.max [1/s]:   0.010000   0.010000
snv t:         0 h:     0.48 +/-     0.64     0.05 -     3.00 v:     0.00 -     4.08
snv t:         2 h:     0.70 +/-     0.71     0.14 -     3.00 v:     0.01 -     1.95
snv t:         4 h:     0.87 +/-     0.72     0.26 -     3.00 v:     0.01 -     1.35
snv t:         

----